In [1]:
! pip install selenium
! pip install msedge-selenium-tools
! pip install bs4

Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the 'C:\Program Files\Python310\python.exe -m pip install --upgrade pip' command.


Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the 'C:\Program Files\Python310\python.exe -m pip install --upgrade pip' command.


Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the 'C:\Program Files\Python310\python.exe -m pip install --upgrade pip' command.


In [2]:
! pip install webdriver_manager

Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the 'C:\Program Files\Python310\python.exe -m pip install --upgrade pip' command.


In [1]:
from selenium import webdriver
# import chromedriver_binary
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import csv

#### Experimentation 

In [9]:
# driver = webdriver.Chrome()
driver = webdriver.Chrome(ChromeDriverManager().install())

In [41]:
# url= 'https://www.amazon.com/'

In [42]:
# driver.get(url)

In [10]:
def my_url(keyword):
    temp= 'https://www.amazon.com/s?k={}&ref=nav_bb_sb'
    keyword= keyword.replace(' ','+')
    return temp.format(keyword)

In [11]:
url= my_url('laptop charger')

driver.get(url)

In [12]:

soup = BeautifulSoup(driver.page_source,'html.parser')
soup_results=soup.find_all('div',{'data-component-type':'s-search-result'})

In [13]:
len(soup_results)

22

In [14]:
obj=soup_results[0]

In [19]:
# rate=obj.find('span','a-icon-alt').text
# rate
count_review = obj.find('span',{'class':'a-size-base s-underline-text'}).text
count_review

'411'

#### Generic function

In [20]:
def my_url(keyword):
    temp= 'https://www.amazon.com/s?k={}&ref=nav_bb_sb'
    keyword= keyword.replace(' ','+')
    url= temp.format(keyword)

    url += '&page{}'   # multiple pages , will be changed in ref statement in link

    return url


def extract_records(obj):
    atag= obj.h2.a
    des= atag.text.strip()
    url = 'https://amazon.com/'+ atag.get('href')
    #it is possible that some items on amazom.com might not be having one of the items we are looking for(e.g. some items might not be having ratings or price), 
    # we will be getting error if we dont take care of that. We will therefore add some error handlers
    #if there are no price,probably the item is out of stock or not available, 
    # then we will ignore the item, but if there are no reviews yet, 
    # it's fine, we will still want to extract the item.
    try:
       parent = obj.find('span','a-price')
       price= parent.find('span','a-offscreen').text  # if the price is not there then through an exception but don't stop
    except AttributeError:
        return 
    
    try:
       rate= obj.find('span','a-icon-alt').text 
       count_review = obj.find('span',{'class':'a-size-base s-underline-text'}).text # obj.find('span',{'class':'a-size-base','dir':'auto'}).text
       
    except AttributeError:
        rate = ''
        count_review=''
    image =obj.find('img',{'class':'s-image'}).get('src')

    result= (des, price,rate, count_review,url,image)
    return result

def main(keyword):
    driver = webdriver.Chrome(ChromeDriverManager().install())

    records = []
    url= my_url(keyword)
    
    for page in range(1,3):
        driver.get(url.format(page))

        soup= BeautifulSoup(driver.page_source,'html.parser')      
        soup_result= soup.find_all('div',{'data-component-type': 's-search-result'})

        for item in soup_result:
            record = extract_records(item)
            if record :
                records.append(record)
        with open('result.csv','w',newline='',encoding='utf-8') as f:
            writer = csv.writer(f)
            writer.writerow(['Description', 'Price','Rate', 'Review count','URL','Image']) 
            writer.writerows(records)

In [21]:
main('pencil box')

In [4]:
import pandas as pd

In [22]:
df=pd.read_csv(r'C:\Users\aeshna_gupta\OneDrive - Dell Technologies\DGA\Udemy courses\Web scrapping\result.csv')
df

,Description,Price,Rate,Review count,URL,Image
0,"Rabbit Pencil Case for Girls, 3D Pen Pouch Sta...",$9.09,5.0 out of 5 stars,9,https://amazon.com//sspa/click?ie=UTF8&spc=MTo...,https://m.media-amazon.com/images/I/71JBBtGAR8...
1,1 Pack Plastic Pencil Box Large Capacity Penci...,$5.99,4.4 out of 5 stars,179,https://amazon.com//sspa/click?ie=UTF8&spc=MTo...,https://m.media-amazon.com/images/I/618cuS0cqV...
2,"Mr. Pen- Pencil Box, Assorted Color, 2 Pack, P...",$5.99,4.2 out of 5 stars,14,https://amazon.com//sspa/click?ie=UTF8&spc=MTo...,https://m.media-amazon.com/images/I/81rdG7t+XL...
3,"YOKUMA Pencil Case,Large Capacity Pencil Pouch...",$8.99,4.6 out of 5 stars,249,https://amazon.com//sspa/click?ie=UTF8&spc=MTo...,https://m.media-amazon.com/images/I/8173RezFVa...
4,"Large Capacity Clear Pencil Box, Pencil Case f...",$4.99,4.7 out of 5 stars,60,https://amazon.com//Capacity-Pencil-Plastic-St...,https://m.media-amazon.com/images/I/51GjzKiDDf...
...,...,...,...,...,...,...
109,Hard Shell Molded Zippered Pencil/Storage Case...,$10.99,4.6 out of 5 stars,250,https://amazon.com//Molded-Zippered-Pencil-Sto...,https://m.media-amazon.com/images/I/71-iQ2ca9y...
110,"1InTheoffice Pencil Box, Translucent Clear (6 ...",$17.99,4.4 out of 5 stars,237,https://amazon.com//1InTheoffice-Pencil-Transl...,https://m.media-amazon.com/images/I/61+0jfdZVn...
111,Pencil Box 3 Pack Small Color Multi Purpose Pe...,$9.99,4.6 out of 5 stars,274,https://amazon.com//Purpose-Supplies-Durable-P...,https://m.media-amazon.com/images/I/71RLorkjQ3...
112,"Tribello Pencil Box for Kids, Universal School...",$9.99,4.7 out of 5 stars,212,https://amazon.com//Tribello-Pencil-Universal-...,https://m.media-amazon.com/images/I/61btpNNIxG...
